In [1]:
!pip install transformers datasets sacrebleu torch evaluate
!pip install codebleu
!pip install tree-sitter-python
!git clone https://github.com/microsoft/CodeXGLUE.git
%cd CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU
!pip install tree_sitter==0.2.0
#build parser
%cd parser
!git clone https://github.com/tree-sitter/tree-sitter-php
!bash build.sh
%cd /content


from sklearn.metrics import f1_score
import pandas as pd
import re
import torch
from transformers import RobertaTokenizer
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments, EarlyStoppingCallback
from evaluate import load as load_metric
import subprocess


  Using cached tree_sitter-0.22.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
Using cached tree_sitter-0.22.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (544 kB)
  Attempting uninstall: tree-sitter
    Found existing installation: tree_sitter 0.2.0
    Uninstalling tree_sitter-0.2.0:
      Successfully uninstalled tree_sitter-0.2.0
fatal: destination path 'CodeXGLUE' already exists and is not an empty directory.
/content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU
  Using cached tree_sitter-0.2.0-cp311-cp311-linux_x86_64.whl
  Attempting uninstall: tree_sitter
    Found existing installation: tree-sitter 0.22.3
    Uninstalling tree-sitter-0.22.3:
      Successfully uninstalled tree-sitter-0.22.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
codebleu 0.7.0 requires tree-sitter<0.23.0,>=0.22.0, but 

In [12]:
#---------- Load and Prepare Data ----------#

train = pd.read_csv("/content/ft_train.csv")
valid = pd.read_csv("/content/ft_valid.csv")
test = pd.read_csv("/content/ft_test.csv")

def mask_if_condition(row):
    """Replace the target_block line in cleaned_method with <mask> and flatten the function."""
    target = row['target_block'].strip()
    method = row['cleaned_method']

    # Escape special characters in regex pattern
    escaped_target = re.escape(target)

    # Replace only the first occurrence of the if statement with <mask>
    masked_method = re.sub(escaped_target, '<mask>', method, count=1)

    # Flatten: Remove newlines, tabs, and excessive spaces
    flattened = " ".join(masked_method.strip().split())

    return flattened

# Apply to all datasets
for df in [train, valid, test]:
    df['masked_method'] = df.apply(mask_if_condition, axis=1)


# Preview one
train[['cleaned_method', 'target_block', 'masked_method']].head()

,cleaned_method,target_block,masked_method
0,"def _resolve_lib_imported_symbols(self, lib, i...",if generic_refs :,"def _resolve_lib_imported_symbols(self, lib, i..."
1,"def make_docs_directory(output_dir, name):\n ...","if not isdir ( pjoin ( output_dir , name , str...","def make_docs_directory(output_dir, name): if ..."
2,"def assert_results(self, results, activities, ...","if hasattr ( result , ""extra_context"" ) :","def assert_results(self, results, activities, ..."
3,"def for_file(cls, filename: str, modname: str)...","if "".egg"" + path . sep in filename :","def for_file(cls, filename: str, modname: str)..."
4,"def merge_dicts(source: Dict, destination: Dic...","if isinstance ( value , dict ) :","def merge_dicts(source: Dict, destination: Dic..."


In [13]:
#---------- Tokenization and Data Wrapping ----------#

tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-small")

# Constants
MAX_INPUT_LEN = 512
MAX_TARGET_LEN = 64

# Custom dataset for fine-tuning CodeT5
class CodeIfDataset(Dataset):
    def __init__(self, df, tokenizer, source_col='masked_method', target_col='target_block'):
        self.inputs = df[source_col].tolist()
        self.targets = df[target_col].tolist()
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_text = self.inputs[idx]
        target_text = self.targets[idx]

        input_enc = self.tokenizer(
            input_text,
            padding="max_length",
            truncation=True,
            max_length=MAX_INPUT_LEN,
            return_tensors="pt"
        )
        target_enc = self.tokenizer(
            target_text,
            padding="max_length",
            truncation=True,
            max_length=MAX_TARGET_LEN,
            return_tensors="pt"
        )

        return {
            "input_ids": input_enc.input_ids.squeeze(),
            "attention_mask": input_enc.attention_mask.squeeze(),
            "labels": target_enc.input_ids.squeeze()
        }

# Create Dataset objects
train_dataset = CodeIfDataset(train, tokenizer)
valid_dataset = CodeIfDataset(valid, tokenizer)
test_dataset = CodeIfDataset(test, tokenizer)

In [14]:
#---------- Finetune Model ----------#

# Load the CodeT5-small model
model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-small")

# Define training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/CodeBLEU_Project/codet5-if-checkpoints",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=7,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    logging_dir="./logs",
    report_to="none"  # Set to "wandb" or "tensorboard" if you want visual logs
)

# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

# Start training
trainer.train()
#Resume training
#trainer.train(resume_from_checkpoint= True)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


Epoch,Training Loss,Validation Loss


TrainOutput(global_step=43750, training_loss=0.0, metrics={'train_runtime': 0.2729, 'train_samples_per_second': 1282435.429, 'train_steps_per_second': 160304.429, 'total_flos': 4.73696305152e+16, 'train_loss': 0.0, 'epoch': 7.0})

In [15]:
def evaluate_and_export_results(df, output_csv="testset-results.csv"):

    # Check required columns
    required_cols = ["input", "Predicted full method", "Expected full method"]
    if not all(col in df.columns for col in required_cols):
        raise ValueError(f"DataFrame must contain columns: {required_cols}")

    #Compute F1 Score
    y_true = [1] * len(df)
    y_pred = [int(p.strip() == r.strip()) for p, r in zip(df["Predicted if condition"], df["Expected if condition"])]
    f1 = round(f1_score(y_true, y_pred) * 100, 2)
    print(f"F1 Score (Exact Match): {f1}%")

    # Compute exact match
    df["Exact match"] = df["Predicted full method"].str.strip() == df["Expected full method"].str.strip()

    # Save refs and hyps to temp files for CodeBLEU
    df["Expected full method"].to_csv("ref.txt", index=False, header=False)
    df["Predicted full method"].to_csv("hyp.txt", index=False, header=False)

    # Load BLEU scorer
    bleu = load_metric("sacrebleu")

    # BLEU-4 score
    predictions = df["Predicted full method"].tolist()
    references = [[ref] for ref in df["Expected full method"].tolist()]
    bleu_score = bleu.compute(predictions=predictions, references=references)
    bleu_score_value = round(bleu_score["score"], 2)

    # Run CodeBLEU using subprocess
    try:
        result = subprocess.check_output(
            ["python", "calc_codebleu.py", "--ref", "ref.txt", "--hyp", "hyp.txt", "--lang", "python"],
            cwd="/content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU",
            text=True
        )
        for line in result.splitlines():
            if "CodeBLEU score" in line:
                codebleu_score_value = round(float(line.split(":")[-1].strip()), 2)
                break
        else:
            codebleu_score_value = 0.
    except Exception as e:
        codebleu_score_value = 0.0
        print("CodeBLEU failed:", e)



    # Add scores to DataFrame
    df["BLEU-4 prediction score (0-100)"] = bleu_score_value
    df["CodeBLEU prediction score (0-100)"] = codebleu_score_value

    # Export final CSV
    df[[
        "input",
        "Exact match",
        "Expected if condition",
        "Predicted if condition",
        "CodeBLEU prediction score (0-100)",
        "BLEU-4 prediction score (0-100)"
    ]].to_csv(output_csv, index=False)

    print(f"Evaluation complete. Results saved to {output_csv}")
    return output_csv

In [16]:
#---------- Evaluation and Prediction ----------#

#Make Predicitons
model.eval() # Put model in evaluation mode

predictions = []
references = []
inputs = []

# Run prediction loop
for batch in DataLoader(test_dataset, batch_size=8):
    input_ids = batch["input_ids"].to(model.device)
    attention_mask = batch["attention_mask"].to(model.device)

    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=64)

    decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(batch["labels"], skip_special_tokens=True)
    original_inputs = tokenizer.batch_decode(input_ids, skip_special_tokens=True)

    #strips white space
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]
    original_inputs = [inp.strip() for inp in original_inputs]

    predictions.extend(decoded_preds)
    references.extend(decoded_labels)
    inputs.extend(original_inputs)

test_df = pd.DataFrame({
    "input": inputs,                             # masked method with <mask>
    "Expected if condition": references,         # the original target_block
    "Predicted if condition": predictions,       # model's generated if
    "Expected full method": [inp.replace("<mask>", ref) for inp, ref in zip(inputs, references)],
    "Predicted full method": [inp.replace("<mask>", pred) for inp, pred in zip(inputs, predictions)],
})


In [17]:
evaluate_and_export_results(test_df)


F1 Score (Exact Match): 78.82%
CodeBLEU failed: Command '['python', 'calc_codebleu.py', '--ref', 'ref.txt', '--hyp', 'hyp.txt', '--lang', 'python']' returned non-zero exit status 2.
Evaluation complete. Results saved to testset-results.csv


'testset-results.csv'